In [34]:
# ==================================================================================
# TITAN VALIDATION FRAMEWORK (TVF) — OMNI EDITION (FINAL)
# Role: Senior Data Architect
# Capabilities: 
#   1. DE Ops (Schema, Volume, Dupes)      2. Statistical (Num/Cat Drift)
#   3. Structural (PCA, Manifold)          4. Causal (Stability, Paradox)
#   5. Logic/Business Rules                6. Forensics (Benford)
#   7. Ethics (Fairness/Bias)              8. Security (Leakage)
# ==================================================================================

import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import logging
from dataclasses import dataclass
from typing import Dict, List, Tuple
from scipy.stats import ks_2samp, chisquare
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1. CONFIGURATION
# -----------------------------
@dataclass
class TVFConfig:
    # Statistical
    drift_alpha: float = 0.05
    reconstruction_error_threshold: float = 0.05 
    min_explained_variance_ratio: float = 0.90 
    
    # Integrity & Ops
    max_null_spike: float = 0.10
    min_numeric_coercion: float = 0.98
    max_volumetric_drift: float = 0.50  # Alert if row count changes by +/- 50%
    max_duplicate_rate: float = 0.0     # Zero tolerance for dupes
    
    # Forensics
    enable_benfords_law: bool = True
    
    # Causal & Ethics
    max_correlation_drift: float = 0.25
    simpsons_threshold: float = 0.3
    max_fairness_disparity: float = 0.25 # Max 25% diff in target mean between groups
    
    # Compute
    max_cardinality: int = 100
    allow_zero_variance: bool = False
    max_leakage_r2: float = 0.98
    
    # Temporal
    max_recency_days: int = 30 
    
    # Unsupervised
    enable_iforest: bool = True
    iforest_contamination: float = 0.02

# -----------------------------
# 2. THE ENGINE
# -----------------------------
class TitanValidationFramework:
    def __init__(self, reference_data: pd.DataFrame, config: TVFConfig = None):
        self.logger = logging.getLogger("TVF")
        if not self.logger.handlers:
            self.logger.setLevel(logging.INFO)
            self.logger.addHandler(logging.StreamHandler())
            
        self.config = config or TVFConfig()
        self.reference = reference_data.copy()
        
        # Learn Baseline
        self.num_cols = self.reference.select_dtypes(include=[np.number]).columns.tolist()
        self.cat_cols = self.reference.select_dtypes(include=['object', 'category']).columns.tolist()
        self.ref_corr = self.reference[self.num_cols].corr()
        self.ref_nulls = self.reference.isna().mean()
        self.ref_count = len(self.reference)
        
        self.ref_cat_freqs = {c: self.reference[c].value_counts(normalize=True) for c in self.cat_cols}
        
        # Train Models
        self._train_unsupervised()
        self._train_structure()
        
        self.logger.info(f"TVF Omni Online. Baseline: {self.ref_count} rows.")

    def _train_unsupervised(self):
        self.iforest = None
        if self.config.enable_iforest and len(self.reference) > 50:
            self.iforest = IsolationForest(contamination=self.config.iforest_contamination, random_state=42, n_jobs=-1)
            self.iforest.fit(self.reference[self.num_cols].fillna(0))

    def _train_structure(self):
        self.scaler = StandardScaler()
        X = self.reference[self.num_cols].fillna(0)
        X_scaled = self.scaler.fit_transform(X)
        self.pca = PCA(n_components=0.95)
        self.pca.fit(X_scaled)
        
        X_recon = self.pca.inverse_transform(self.pca.transform(X_scaled))
        self.ref_recon_error = np.mean(np.square(X_scaled - X_recon))
        self.ref_explained_var = np.sum(np.var(self.pca.transform(X_scaled), axis=0)) / np.sum(np.var(X_scaled, axis=0))

    def validate(self, new_data: pd.DataFrame, target_col: str = None, date_col: str = None, 
                 consistency_rules: List[str] = None, subgroups: List[str] = None) -> Tuple[bool, Dict]:
        
        report = {"modules": {}, "traffic_light": None}
        
        # --- SCANS ---
        report["modules"]["integrity"] = self._scan_integrity(new_data)
        report["modules"]["ops"] = self._scan_ops(new_data)
        report["modules"]["drift_num"] = self._scan_drift_numeric(new_data)
        report["modules"]["drift_cat"] = self._scan_drift_categorical(new_data)
        report["modules"]["forensics"] = self._scan_forensics(new_data)
        report["modules"]["structure"] = self._scan_structure(new_data)
        report["modules"]["stability"] = self._scan_stability(new_data)
        report["modules"]["paradox"] = self._scan_simpsons(new_data, target_col, subgroups)
        report["modules"]["fairness"] = self._scan_fairness(new_data, target_col, subgroups)
        report["modules"]["logic"] = self._scan_logic(new_data, consistency_rules)
        report["modules"]["temporal"] = self._scan_temporal(new_data, date_col)
        report["modules"]["compute"] = self._scan_compute(new_data)
        report["modules"]["leakage"] = self._scan_leakage(new_data, target_col)
        report["modules"]["anomalies"] = self._scan_anomalies(new_data)

        # Verdict
        report["traffic_light"] = self._generate_report(report)
        failed = len(report["traffic_light"][report["traffic_light"]["Status"] == "RED"]) > 0
        return (not failed), report

    # --- SCANNERS ---
    def _scan_ops(self, df):
        issues = []
        ratio = len(df) / (self.ref_count + 1e-9)
        if abs(1 - ratio) > self.config.max_volumetric_drift:
            issues.append(f"Volumetric Drift (Size Ratio: {ratio:.2f}x)")
        dupe_rate = df.duplicated().mean()
        if dupe_rate > self.config.max_duplicate_rate:
            issues.append(f"Duplicate Rows ({dupe_rate:.1%})")
        return {"issues": issues}

    def _scan_fairness(self, df, target, groups):
        if not target or not groups or target not in df.columns: return {"issues": []}
        issues = []
        for g in groups:
            if g not in df.columns: continue
            means = df.groupby(g)[target].mean()
            if len(means) < 2: continue
            disparity = (means.max() - means.min()) / (abs(means.min()) + 1e-9)
            if disparity > self.config.max_fairness_disparity:
                issues.append(f"Fairness Warning on '{g}' (Disparity: {disparity:.1%})")
        return {"issues": issues}

    def _scan_drift_categorical(self, df):
        drifted = []
        for c in self.cat_cols:
            if c not in df.columns: continue
            ref_freq = self.ref_cat_freqs.get(c)
            new_counts = df[c].value_counts()
            common_cats = ref_freq.index.intersection(new_counts.index)
            if len(common_cats) < 2: continue
            obs = new_counts[common_cats].sort_index().values
            exp = ref_freq[common_cats].sort_index().values * len(df)
            exp = exp * (obs.sum() / exp.sum())
            try:
                if chisquare(f_obs=obs, f_exp=exp)[1] < self.config.drift_alpha: 
                    drifted.append(c)
            except: pass
        return {"drifted": drifted}

    def _scan_forensics(self, df):
        if not self.config.enable_benfords_law: return {"suspicious": []}
        suspicious = []
        benford_probs = np.log10(1 + 1/np.arange(1, 10))
        for c in self.num_cols:
            if c not in df.columns: continue
            try:
                digits = df[c].astype(str).str.lstrip('-').str[0]
                digits = digits[digits.isin([str(i) for i in range(1, 10)])].astype(int)
                if len(digits) < 100: continue
                counts = digits.value_counts().sort_index()
                obs = np.array([counts.get(i, 0) for i in range(1, 10)])
                if chisquare(obs, f_exp=benford_probs * len(digits))[1] < 0.001: 
                    suspicious.append(f"{c} (Benford Viol.)")
            except: pass
        return {"suspicious": suspicious}

    def _scan_structure(self, df):
        try:
            X = self.scaler.transform(df[self.num_cols].fillna(0))
            X_recon = self.pca.inverse_transform(self.pca.transform(X))
            ratio = np.mean(np.square(X - X_recon)) / max(self.ref_recon_error, 1e-6)
            var_retention = (np.sum(np.var(self.pca.transform(X), axis=0)) / np.sum(np.var(X, axis=0))) / self.ref_explained_var
            if ratio > (1 + self.config.reconstruction_error_threshold):
                return {"drifted": True, "msg": f"Structure Shift (Error Ratio: {ratio:.2f}x)"}
            if var_retention < self.config.min_explained_variance_ratio:
                return {"drifted": True, "msg": f"Structure Collapse (Var Retention: {var_retention:.1%})"}
            return {"drifted": False}
        except Exception as e: return {"drifted": False, "msg": str(e)}

    def _scan_integrity(self, df):
        issues = []
        missing = set(self.reference.columns) - set(df.columns)
        if missing: issues.append(f"Missing Columns: {list(missing)[:3]}...")
        for c in df.columns:
            if (df[c].isna().mean() - self.ref_nulls.get(c, 0)) > self.config.max_null_spike:
                issues.append(f"{c}: Null Spike")
        return {"issues": issues}

    def _scan_drift_numeric(self, df):
        drifted = []
        for c in self.num_cols:
            if c not in df.columns: continue
            try:
                if ks_2samp(self.reference[c].dropna(), df[c].dropna())[1] < self.config.drift_alpha:
                    drifted.append(c)
            except: pass
        return {"drifted": drifted}

    def _scan_stability(self, df):
        common = [c for c in self.num_cols if c in df.columns]
        if len(common) < 2: return {"broken": []}
        diff = (self.reference[common].corr() - df[common].corr()).abs()
        broken = []
        for i in range(len(common)):
            for j in range(i+1, len(common)):
                if diff.iloc[i, j] > self.config.max_correlation_drift:
                    broken.append(f"{common[i]}-{common[j]}")
        return {"broken": broken}

    def _scan_simpsons(self, df, target, groups):
        reversals = []
        if not target or not groups: return {"reversals": []}
        feats = [f for f in self.num_cols if f != target and f in df.columns]
        for g in groups:
            if g not in df.columns: continue
            for f in feats:
                try:
                    g_corr = df[f].corr(df[target])
                    if abs(g_corr) < self.config.simpsons_threshold: continue
                    for sub in df[g].unique():
                        sub_df = df[df[g] == sub]
                        if len(sub_df) < 30: continue
                        s_corr = sub_df[f].corr(sub_df[target])
                        if (np.sign(g_corr) != np.sign(s_corr)) and abs(s_corr) > self.config.simpsons_threshold:
                            reversals.append(f"{f} (Global {g_corr:.2f}, {sub} {s_corr:.2f})")
                            break
                except: pass
        return {"reversals": reversals}

    def _scan_logic(self, df, rules):
        violations = []
        if rules:
            for r in rules:
                try:
                    if (~df.eval(r).fillna(False)).sum() > 0: violations.append(f"Rule Failed: {r}")
                except: pass
        return {"violations": violations}

    def _scan_temporal(self, df, date_col):
        issues = []
        if date_col and date_col in df.columns:
            try:
                dates = pd.to_datetime(df[date_col])
                if not dates.is_monotonic_increasing: issues.append("Time Travel (Unsorted)")
                if (pd.Timestamp.now() - dates.max()).days > self.config.max_recency_days:
                    issues.append("Stale Data")
            except: issues.append("Date Parse Failed")
        return {"issues": issues}

    def _scan_compute(self, df):
        issues = []
        for c in df.select_dtypes(include=['object', 'category']):
            if df[c].nunique() > self.config.max_cardinality: issues.append(f"{c}: High Cardinality")
        if not self.config.allow_zero_variance:
            for c in self.num_cols:
                if c in df.columns and df[c].var() == 0: issues.append(f"{c}: Zero Variance")
        return {"issues": issues}

    def _scan_leakage(self, df, target):
        leaks = []
        if target and target in df.columns:
            y = df[target].fillna(0)
            for c in self.num_cols:
                if c == target or c not in df.columns: continue
                try:
                    if LinearRegression().fit(df[[c]].fillna(0), y).score(df[[c]].fillna(0), y) > self.config.max_leakage_r2:
                        leaks.append(c)
                except: pass
        return {"leaks": leaks}

    def _scan_anomalies(self, df):
        if not self.iforest: return {"rate": 0.0, "status": "SKIPPED"}
        rate = (self.iforest.predict(df[self.num_cols].fillna(0)) == -1).mean()
        return {"rate": rate, "status": "RED" if rate > self.config.iforest_contamination * 4 else "GREEN"}

    def _generate_report(self, report):
        rows = []
        map_ = {
            "integrity": "Integrity", "ops": "Ops Health", "drift_num": "Numeric Drift", 
            "drift_cat": "Categorical Drift", "structure": "Multivariate Drift", 
            "stability": "Causal Stability", "paradox": "Paradox", "fairness": "Algorithmic Fairness",
            "logic": "Business Logic", "temporal": "Temporal", "compute": "Compute", 
            "leakage": "Data Leakage", "anomalies": "Multivariate Anomaly", "forensics": "Forensics"
        }
        for mod, label in map_.items():
            data = report["modules"].get(mod, {})
            errs = []
            for k in ["issues", "drifted", "broken", "violations", "reversals", "leaks", "suspicious"]:
                if isinstance(data.get(k), list): errs.extend(data[k])
            if mod == "anomalies" and data.get("status") == "RED": errs.append(f"Anomaly Rate {data['rate']:.1%}")
            if mod == "structure" and data.get("drifted"): errs.append(data.get("msg"))
            
            for e in errs:
                rows.append({"Module": label, "Feature": e.split(":")[0] if ":" in str(e) else "Global", "Status": "RED", "Reason": str(e)})
                
        return pd.DataFrame(rows).sort_values("Module") if rows else pd.DataFrame([{"Status": "GREEN", "Reason": "All Checks Passed"}])

# ==================================================================================
# 3. TITAN OMNI GAUNTLET: The "No Gaps" Proof (CORRECTED SEQUENCE)
# ==================================================================================
if __name__ == "__main__":
    from sklearn.datasets import load_diabetes
    print("--- 1. INITIALIZING TITAN OMNI ---")
    raw = load_diabetes(as_frame=True, scaled=False)
    df_ref = raw.frame
    df_ref["Date"] = pd.date_range(end=pd.Timestamp.now(), periods=len(df_ref))
    df_ref["Region"] = np.random.choice(["A", "B"], size=len(df_ref)) 
    
    titan = TitanValidationFramework(df_ref)

    # 4. GENERATE 5-PRONGED ATTACK
    print("\n--- 2. GENERATING 'HYDRA' ATTACK DATA ---")
    df_attack = df_ref.copy()
    
    # Attack 1: Volumetric (Drop 60% of data) -> Triggers >50% Threshold
    df_attack = df_attack.sample(frac=0.4, random_state=42)
    
    # Attack 2: Fairness (Bias)
    mask_a = df_attack["Region"] == "A"
    df_attack.loc[mask_a, "target"] *= 2.0
    
    # Attack 4: Forensics (Benford)
    # *CRITICAL*: Add this column BEFORE creating duplicates, otherwise random values make rows unique!
    df_attack["Expense"] = np.random.randint(100, 999, size=len(df_attack))
    
    # Attack 3: Duplicates
    # Append 20 duplicates AFTER all columns are set
    df_attack = pd.concat([df_attack, df_attack.iloc[:20]])
    
    # Attack 5: Logic
    df_attack.loc[df_attack.index[:5], "bmi"] = -10.0

    # 5. EXECUTE SCAN
    print("\n--- 3. RUNNING OMNI SCAN ---")
    safe, report = titan.validate(
        df_attack, 
        target_col="target", 
        date_col="Date", 
        consistency_rules=["bmi > 0"], 
        subgroups=["Region"]
    )
    
    # 6. VERIFY RESULTS
    print(f"\n[FINAL VERDICT]: {'✅ SAFE' if safe else '🛑 UNSAFE'}")
    failures = report["traffic_light"][report["traffic_light"]["Status"] == "RED"]
    
    if not failures.empty:
        print("\n" + failures.to_string(index=False))
    
    # 7. PROOF
    print("\n[OMNI PROOF]")
    reasons = str(failures["Reason"].values)
    features = str(failures["Feature"].values)
    
    checks = {
        "Volume": "Volumetric" in reasons,
        "Fairness": "Fairness" in reasons,
        "Duplicates": "Duplicate" in reasons,
        "Benford": "Benford" in reasons or "Expense" in features,
        "Logic": "bmi > 0" in reasons
    }
    
    for name, result in checks.items():
        print(f"{name:<12} Detected? {'✅ YES' if result else '❌ NO'}")
        
    if all(checks.values()):
        print("\n🚀 TITAN OMNI: ALL 5 ATTACK VECTORS NEUTRALIZED. ZERO GAPS CONFIRMED.")
    else:
        print("\n⚠️ WARNING: Some attacks slipped through.")

--- 1. INITIALIZING TITAN OMNI ---


TVF Omni Online. Baseline: 442 rows.



--- 2. GENERATING 'HYDRA' ATTACK DATA ---

--- 3. RUNNING OMNI SCAN ---

[FINAL VERDICT]: 🛑 UNSAFE

              Module                                 Feature Status                                          Reason
Algorithmic Fairness Fairness Warning on 'Region' (Disparity    RED Fairness Warning on 'Region' (Disparity: 96.2%)
      Business Logic                             Rule Failed    RED                            Rule Failed: bmi > 0
    Causal Stability                                  Global    RED                                         age-bmi
    Causal Stability                                  Global    RED                                          bmi-s1
    Causal Stability                                  Global    RED                                          bmi-s2
    Causal Stability                                  Global    RED                                          bmi-s4
    Causal Stability                                  Global    RED                    